# Three Link Acrobot Controller

This code contains only the Xin swing-up controller without an attempt to switch over to LQR in the region of attraction. 

## Notebook Setup 
The following cell will install Drake, checkout the underactuated repository, and set up the path (only if necessary).
- On Google's Colaboratory, this **will take approximately two minutes** on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  Colab will ask you to "Reset all runtimes"; say no to save yourself the reinstall.
- On Binder, the machines should already be provisioned by the time you can run this; it should return (almost) instantly.

More details are available [here](http://underactuated.mit.edu/drake.html).

In [0]:
try:
    import pydrake
    import underactuated
except ImportError:
    !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
    from jupyter_setup import setup_underactuated
    setup_underactuated()

# Setup matplotlib.  
from IPython import get_ipython
if get_ipython() is not None: get_ipython().run_line_magic("matplotlib", "inline")

In [0]:
# python libraries
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

# pydrake imports
from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder, Parser,
                         PlanarSceneGraphVisualizer, Simulator, VectorSystem,
                         Multiplexer, MatrixGain, LogOutput, ConstantVectorSource, 
                         Saturation, WrapToSystem, wrap_to, LinearQuadraticRegulator, SceneGraph, 
                        MultibodyPlant, SignalLogger, Linearize)

from pydrake.systems.drawing import plot_system_graphviz
from pydrake.examples.acrobot import AcrobotSpongController
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import InputPortIndex, InputPortSelection, OutputPortSelection

# underactuated imports
from underactuated import FindResource, ManipulatorDynamics

from google.colab import files
uploaded = files.upload()



In [0]:
import os
from underactuated.jupyter import AdvanceToAndVisualize, SetupMatplotlibBackend
plt_is_interactive = SetupMatplotlibBackend()



In [0]:
import copy

#Lists for keeping track of variables
Vs = []
Es = []
PEs = []
KEs = []
qabars = []
Vs = []

class XinController(VectorSystem):
    
    def __init__(self, double_pendulum):
        
        # 4 inputs: double pendulum state
        # 2 outputs: elbow torque
        VectorSystem.__init__(self, 6,  2)
        
        
        self.double_pendulum = double_pendulum
        self.double_pendulum_context = double_pendulum.CreateDefaultContext()
        self.double_pendulum_context.SetContinuousState([np.pi, 0, 0, 0, 0, 0])
        
        # Potential energy corresponding to upright equilibrium state
        self.E_desired = double_pendulum.EvalPotentialEnergy(self.double_pendulum_context)
        print(self.E_desired)
        
        # Determine betas from the gravitational terms
        self.double_pendulum_context.SetContinuousState([np.pi/2, 0, 0, 0, 0, 0])
        self.G = -self.double_pendulum.CalcGravityGeneralizedForces(self.double_pendulum_context)
        self.double_pendulum_context.SetContinuousState([0, 0, 0, 0, 0, 0])
        self.beta_3 = self.G[2]
        self.beta_2 = self.G[1] - self.G[2]
        self.beta_1 = self.G[0] - self.G[1]

        # Choose controller gains such that the code doesn't hit singularities
        kp = max(2*self.beta_1*(self.beta_2 + self.beta_3), 2*self.beta_2*self.beta_3*(self.E_desired)/(self.beta_2 + self.beta_3))
        self.k_v = 20000 
        self.k_p = 1.1*kp 
        print(self.k_p)
        self.k_d = 500 
        print('Done w/ Initialization')
    
    def CalcQbar(self, q2, q3):
        beta2_bar = np.sqrt(
            self.beta_2**2 + self.beta_3**2 \
            + 2*self.beta_2*self.beta_3*np.cos(q3))
        sin_theta_q3 = self.beta_3*np.sin(q3)/beta2_bar
        theta_q3 = np.arcsin(sin_theta_q3)
        cos_theta_q3 = (self.beta_2 + self.beta_3*np.cos(q3))/beta2_bar
        theta_q3_2 = np.arccos(cos_theta_q3)
        if theta_q3 < 0:
          thetas2 = [round(theta_q3_2,5), round(-theta_q3_2,5) , round(2*np.pi - theta_q3_2,5)]
          thetas1 = [round(theta_q3,5), round(np.pi - theta_q3,5), round(-np.pi - theta_q3,5)]
          theta_q3 = set(thetas1) & set(thetas2)
          theta_q3 = list(theta_q3)[0]
        q2_bar = q2 + theta_q3
        return [q2_bar, beta2_bar]
 

    def DoCalcVectorOutput(self, context, state, unused, delta):
        
        # Rewrite state vector in terms of qs
        q1 = state[0]
        q2 = state[1]
        q3 = state[2]
        q1dot = state[3]
        q2dot = state[4]
        q3dot = state[5]
        qdot = np.array([[q1dot], [q2dot], [q3dot]])
        qadot = np.array([[q2dot], [q3dot]])

        self.double_pendulum_context.SetContinuousState(state)

        # Calculate mass matrix based on acrobot's current state
        M = self.double_pendulum.CalcMassMatrixViaInverseDynamics(self.double_pendulum_context)
        M11 = M[0, 0]
        M1a = np.array([M[0, 1:]])
        Ma1 = M1a.T
        Maa = np.array([M[1:, 1:]])
        
        # Calculate potential and kinetic energy based on acrobot's current state
        PE = double_pendulum.EvalPotentialEnergy(self.double_pendulum_context)
        KE = double_pendulum.EvalKineticEnergy(self.double_pendulum_context)
        E = PE + KE
        E_tilde = E - self.E_desired

        H = self.double_pendulum.CalcBiasTerm(self.double_pendulum_context)
        H1 = H[0]
        # print('Ha')
        Ha = np.array([H[-2:]]).T
        # print(Ha)
        G = -self.double_pendulum.CalcGravityGeneralizedForces(self.double_pendulum_context)
        G1 = G[0]
        Ga = np.array([G[-2:]]).T
        # print(Ga)
        
        ## Calculate correct q2bar
        res = self.CalcQbar(q2, q3)
        q2_bar = res[0]
        beta2_bar = res[1]

      
        #print('q2bar', q2_bar)
        qa_bar = np.array([[q2_bar], [state[2]]])
        qabars.append(q2_bar)
        

        
        M11_inv = 1/M11
        psi = self.beta_3*(self.beta_3 + self.beta_2*np.cos(state[2]))/(beta2_bar)**2
        Maa_hat = Maa - Ma1.dot(M11_inv).dot(M1a)
        w, v = np.linalg.eig(Maa_hat)
        lambda_max = max(w)
        phi = np.sqrt(self.beta_1**2 + self.beta_2**2 + self.beta_3**2\
                      + 2*self.beta_1*self.beta_2*np.cos(q2)\
                      + 2*self.beta_1*self.beta_3*np.cos(q2 + q3)\
                      + 2*self.beta_2*self.beta_3*np.cos(q3))
        kd_condition = (self.E_desired + phi)*lambda_max*Maa_hat
        kd_condition = np.amax(kd_condition)
        if self.k_d < kd_condition:
            self.k_d = kd_condition
        Ha_hat = np.array([Ha - Ma1.dot(M11_inv).dot(H1)])
        Ga_hat = np.array([Ga - Ma1.dot(M11_inv).dot(G1)])
        Lambda = self.k_d*(np.identity(2)) + (E_tilde)*Maa_hat
        S = np.array([[1, psi], [0, 1]])

        #Check to make sure that Lambda is always invertible 
        if np.abs(np.linalg.det(Lambda)) < 0.01:
          print('Singularity?')
          print(np.linalg.det(Lambda))
        
        lambda_inv = np.linalg.inv(Lambda)

        # Control Law
        tau = lambda_inv.dot(self.k_d*(Ha_hat + Ga_hat) - Maa_hat.dot(self.k_v*qadot + self.k_p*S.T.dot(qa_bar)))

        # Lyapunov Function
        V = (0.5*(E_tilde)**2 + 0.5*self.k_d*qadot.T.dot(qadot) + 0.5*self.k_p*qa_bar.T.dot(qa_bar))[0]
        Vs.append(V)
        # print(tau)
        Es.append(E)
        PEs.append(PE)
        KEs.append(KE)


        # Assign value of controller at each time step
        delta[:] = [tau[0,0], tau[0,1]] 
        
        


In [0]:



builder = DiagramBuilder()

# instantiate the acrobot and the scene graph
# the scene graph is a container for the geometries of all the physical systems in our diagram
double_pendulum, scene_graph = AddMultibodyPlantSceneGraph(
    builder,
    time_step=0. # discrete update period , set to zero since system is continuous
)

# parse the urdf and populate the vibrating pendulum
urdf_path = 'three_link_acrobot.urdf'
Parser(double_pendulum).AddModelFromFile(urdf_path)
double_pendulum.Finalize()


#saturation = builder.AddSystem(Saturation(min_value=[[-10000], [-10000]], max_value=[[10000], [10000]]))
# builder.Connect(saturation.get_output_port(0), double_pendulum.get_actuation_input_port())
wrapangles = WrapToSystem(6)
wrapangles.set_interval(0, 0, 2*np.pi)
wrapangles.set_interval(1, -np.pi, np.pi)
wrapangles.set_interval(2, -np.pi, np.pi)
wrapto = builder.AddSystem(wrapangles)
controller = builder.AddSystem(XinController(double_pendulum))
# builder.Connect(double_pendulum.get_state_output_port(), controller.get_input_port(0))
builder.Connect(double_pendulum.get_state_output_port(), wrapto.get_input_port(0))
builder.Connect(wrapto.get_output_port(0), controller.get_input_port(0))
builder.Connect(controller.get_output_port(0), double_pendulum.get_actuation_input_port())
# builder.Connect(controller.get_output_port(0), saturation.get_input_port(0))

# Comment out to not run visualization
visualizer = builder.AddSystem(
   PlanarSceneGraphVisualizer(scene_graph, xlim=[-4., 4.], ylim=[-4., 4.], show=plt_is_interactive))
builder.Connect(scene_graph.get_pose_bundle_output_port(),
               visualizer.get_input_port(0))

logger = builder.AddSystem(SignalLogger(6))
builder.Connect(wrapto.get_output_port(0), logger.get_input_port(0))
control_logger = builder.AddSystem(SignalLogger(2))
builder.Connect(controller.get_output_port(0), control_logger.get_input_port(0))
diagram = builder.Build()


# Set up a simulator to run this diagram
simulator = Simulator(diagram)
context = simulator.get_mutable_context()

# Simulate
duration = 150.0 if get_ipython() else 0.1 # sets a shorter duration during testing



fig, ax = plt.subplots(6)
fig.set_figheight(10)
fig.set_figwidth(10)
context.SetTime(0.)
context.SetContinuousState([-np.pi/12, 0, 0, 0, 0, 0]) #np.random.randn(4,1))
# simulator.AdvanceTo(duration)
AdvanceToAndVisualize(simulator, visualizer, duration)

# Plot key values
ax[0].plot(logger.data()[0])
ax[0].plot([np.pi]*len(logger.data()[0]))
ax[0].set_xlabel("Time Steps")
ax[0].set_ylabel("Theta_1")
ax[1].plot(Es[:])
ax[1].plot([682]*len(Es[:]))
ax[1].set_xlabel("Time Steps")
ax[1].set_ylabel("E")
ax[2].plot(control_logger.data()[0])
ax[2].set_xlabel("Time Steps")
ax[2].set_ylabel("Tau2s")
ax[3].plot(control_logger.data()[1])
ax[3].set_xlabel("Time Steps")
ax[3].set_ylabel("Tau3s")
ax[4].plot(qabars[:])
ax[4].set_xlabel("Time Steps")
ax[4].set_ylabel("Q2bars")
ax[5].plot(Vs[:])
ax[5].set_xlabel("Time Steps")
ax[5].set_ylabel("V(x)")
plt.show()


